In [ ]:
from find_source import summary
from catalogs import low_level_csv, high_level_csv, light_curve
from obs_html import full_html_and_txt
import glob
from astropy.io import fits

In [ ]:
full_html_and_txt('../data/250611_03:56:34')

In [ ]:
#for file in glob.glob('../data/multi_track/*.fits'):
#    try:
#        summary(fits_file=file, short_dict=False, plot=True, save_path='../test')
#    except Exception as e:
#        print(f'Error for {file}: {e}')

In [ ]:
for file in glob.glob('/mnt/COMPASS9/sma/quality/*'):
    try:
        low_level_csv(file)
    except Exception as e:
        print(f'{e} for {file}')

In [ ]:
high_level_csv()

In [ ]:
light_curve()

In [ ]:
#hdul = fits.open('../data/11306/3c273_img_RX3.fits')

In [ ]:
#hdul[1].columns[1].name
#hdul[1].columns[1].unit
#hdul[1].data[1][1]

In [ ]:
#hdul[1].columns

In [ ]:
#float(hdul[1].data[0][2])

In [ ]:
#summary('../data/11306/3c273_img_RX3.fits')